In [ ]:
import random
import math
import matplotlib.pyplot as plt
%matplotlib inline
class SortingNetwork:
    def __init__(self, n, comparators=None):
        self.n = n
        if comparators:
            temp = []
            for a,b in comparators :
                if a != b:
                    temp.append((a,b))
            comparators = temp * 1 #copy temp to d
            self.comparators = comparators
        else:
            self.comparators = [random.sample(range(n), 2) for _ in range(n * (n - 1) // 2)]

    def sort(self, x):
        x = list(x)
        for i, j in self.comparators:
            if x[i] > x[j]:
                x[i], x[j] = x[j], x[i]
        return tuple(x)

    def evaluate(self, inputs):
        errors = 0
        for x in inputs:
            sorted_x = self.sort(x)
            errors += sum(sorted_x[i] > sorted_x[j] for i in range(self.n) for j in range(i + 1, self.n))
        return errors

    def mutate(self):
        mutated_comparators = list(self.comparators)
        i = random.randint(0, len(mutated_comparators) - 1)
        mutated_comparators[i] = random.sample(range(self.n), 2)
        return SortingNetwork(self.n, mutated_comparators)

def generate_inputs(n, num_inputs=1000):
    tuples = []
    for _ in range(num_inputs):
        tpl = random.sample(range(n), n)
        if tpl[0] != tpl[1]:
            tuples.append(tpl)
    return tuples

def simulated_annealing(inputs, initial_temp, cooling_rate, num_iter):
    n = len(inputs[0])
    current_solution = SortingNetwork(n)
    print(len(current_solution.comparators))
    current_energy = current_solution.evaluate(inputs)
    best_solution = current_solution
    best_energy = current_energy
    temperature = initial_temp

    for t in range(1, num_iter + 1):
        temperature *= cooling_rate
        neighbor_solution = current_solution.mutate()
        neighbor_energy = neighbor_solution.evaluate(inputs)
        delta_energy = neighbor_energy - current_energy

        if delta_energy < 0 or random.random() < math.exp(-delta_energy / temperature):
            current_solution = neighbor_solution
            current_energy = neighbor_energy
            print(len(current_solution.comparators))

            if current_energy < best_energy:
                # print(f"Found new solution")
                best_solution = current_solution
                best_energy = current_energy

    return best_solution, best_energy

def generate_inputs(n, num_inputs=100):
    return [tuple(random.sample(range(n), n)) for _ in range(num_inputs)]

def plot_scatter(data, title):
    x, y = zip(*enumerate(data))
    plt.scatter(x, y)
    plt.title(title)
    plt.xlabel('Index')
    plt.ylabel('Value')
    plt.show()

def main(input_data):
    n = len(input_data[0])
    initial_temp = 100
    cooling_rate = 0.999
    num_iter = 5000

    best_solution, best_energy = simulated_annealing(input_data, initial_temp, cooling_rate, num_iter)
    print("Best sorting network found:")
    print(len(best_solution.comparators))
    print("Number of errors:", best_energy)

    # Sort the input data using the optimized sorting network
    sorted_data = [best_solution.sort(x) for x in input_data]

    # Visualize the original and sorted data
    random_input = input_data[random.randint(0, len(input_data) - 1)]
    plot_scatter(random_input, 'Original Data')

    sorted_input = best_solution.sort(random_input)
    print(len(best_solution.comparators))
    print(sorted_input)
    plot_scatter(sorted_input, 'Sorted Data')

    tmp = []
    for a, b in best_solution.comparators:
        if a != b:
            tmp.append((a, b))
    print(len(tmp))

n = 11
num_inputs = 100
input_data = generate_inputs(n, num_inputs)
main(input_data)


In [59]:
import random
import math
import itertools

def remove_duplc(arr):
    tmp = []
    for i, j in arr:
        if i != j and (i, j) not in tmp:
            tmp.append((i, j))
    return tmp

def generate_random_sorting_network(n, num_comparators):
    network = []
    for _ in range(num_comparators):
        i, j = random.sample(range(n), 2)
        if i > j:
            i, j = j, i
        network.append((i, j))
    return network

def apply_sorting_network(network, arr):
    for (i, j) in network:
        if arr[i] > arr[j]:
            arr[i], arr[j] = arr[j], arr[i]

def is_sorting_network(network, n):
    for perm in itertools.permutations(range(n)):
        arr = list(perm)
        apply_sorting_network(network, arr)
        if arr != sorted(arr):
            return False
    return True

def modify_network(network, n):
    idx = random.randint(0, len(network) - 1)
    new_network = list(network)
    i, j = random.sample(range(n), 2)
    if i > j:
        i, j = j, i
    new_network[idx] = (i, j)
    return new_network

def simulated_annealing(network, n, initial_temp, cooling_rate, num_iterations):
    current_temp = initial_temp
    current_network = network

    for _ in range(num_iterations):
        if current_temp <= 0:
            break

        new_network = modify_network(current_network, n)
        if is_sorting_network(new_network, n):
            if len(new_network) < len(current_network) or \
                    math.exp((len(current_network) - len(new_network)) / current_temp) > random.random():
                current_network = new_network

        current_temp *= (1 - cooling_rate)

    return current_network

def optimize_comparators(n, initial_temp=1000, cooling_rate=0.01, num_iterations=1000):
    initial_network = generate_random_sorting_network(n, 50)
    print(len(remove_duplc(initial_network)))
    optimized_network = simulated_annealing(initial_network, n, initial_temp, cooling_rate, num_iterations)
    return optimized_network


n = 10  # Number of elements to sort
optimized_network = optimize_comparators(n)
print("Optimized sorting network:", optimized_network)
print("Optimized sorting network:", remove_duplc(optimized_network))
print("Number of comparators:", len(remove_duplc(optimized_network)))
print("Is a valid sorting network?", is_sorting_network(optimized_network, n))


32
Optimized sorting network: [(0, 7), (0, 4), (5, 8), (4, 9), (0, 6), (0, 5), (5, 6), (7, 9), (5, 7), (2, 7), (0, 9), (3, 7), (1, 8), (0, 3), (2, 6), (4, 9), (6, 7), (5, 7), (3, 5), (4, 7), (2, 8), (4, 6), (2, 5), (5, 8), (0, 2), (7, 8), (2, 7), (3, 4), (1, 3), (4, 7), (5, 7), (5, 6), (5, 6), (1, 9), (8, 9), (2, 7), (6, 8), (0, 6), (3, 5), (0, 7), (1, 4), (5, 6), (4, 7), (0, 6), (7, 8), (1, 5), (3, 4), (4, 5), (7, 8), (1, 7)]
Optimized sorting network: [(0, 7), (0, 4), (5, 8), (4, 9), (0, 6), (0, 5), (5, 6), (7, 9), (5, 7), (2, 7), (0, 9), (3, 7), (1, 8), (0, 3), (2, 6), (6, 7), (3, 5), (4, 7), (2, 8), (4, 6), (2, 5), (0, 2), (7, 8), (3, 4), (1, 3), (1, 9), (8, 9), (6, 8), (1, 4), (1, 5), (4, 5), (1, 7)]
Number of comparators: 32
Is a valid sorting network? False
